In [1]:
import numpy as np
import keyboard
import random
import string
import json
import os
from pythonosc import udp_client
import tkinter as tk
import threading
import keyboard

filename = 'output.json'
ip = "127.0.0.1"
port = 8886

client = udp_client.SimpleUDPClient(ip, port)
# Defining IP and Port of UDP receiver
stop = f"t/stop"
def send_osc_region(index, ip='127.0.0.1', port=8886):
client = udp_client.SimpleUDPClient(ip, port) 

# Reference your own custom action IDs using the REAPER action list here.
commandIDs = [""]

stimulus = f"/action/{commandIDs[index]}"
play = f"t/play"
stop = f"t/stop"

client.send_message(stop, [])
client.send_message(stimulus, [])
client.send_message(play, [])

# Generate stimuli index
stimuli_idx = np.zeros((3,12))
for i in range(0,3):
for j in range(0,12):
    stimuli_idx[i][j] = j + (i * 12)+1
stimuli_idx
def generate_random_id(length=8):

characters = string.ascii_uppercase + string.digits
random_id = ''.join(random.choice(characters) for _ in range(length))
return random_id
def generate_random_order(index):

row_order = []
for num in range(np.shape(index)[0]):
    row_order.append(num)
    
# Randomize row order
np.random.shuffle(row_order)
random_stimuli_order = np.array(index)
for i in range(np.size(row_order)):
    random_stimuli_order[i] = index[row_order[i]].copy()

# Randomize stimulus order within rows
for i in range(np.size(row_order)):
    np.random.shuffle(random_stimuli_order[i])

random_stimuli_order.tolist()

return random_stimuli_order, row_order
def store_user_data(filename, user_id, array_2d):

if isinstance(array_2d, np.ndarray):
    array_2d = array_2d.tolist()


if os.path.exists(filename):
    with open(filename, 'r') as f:
        try:
            data = json.load(f)
        except ValueError:
            data = []
else:
    data = []

# Append new data entry
data.append({'user_id': user_id, 'array_2d': array_2d})

# Write to the file
with open(filename, 'w') as f:
    json.dump(data, f, indent=4)

# Generate the actual user data
arrays, hp_order = generate_random_order(stimuli_idx)
participant_id = generate_random_id()
store_user_data(filename,participant_id, arrays)

# For debugging
print("ID: ",participant_id)

# GUI
def main(text_area):
array_index = 0
value_index = 0
space_press_count = 0 
switch_message_shown = False
waiting_for_final_press = False

def update_text_area(message):
    text_area.insert(tk.END, message + '\n')
    text_area.see(tk.END)

update_text_area(f"Ihre Teilnehmer-ID: {participant_id}")
update_text_area(f"Starte mit Kopfhörer Nr. {hp_order[0] + 1}.")

while array_index < len(arrays):
    keyboard.wait('space')
    space_press_count += 1

    if space_press_count % 2 == 0:
        client.send_message(stop, [])
        update_text_area("Playback gestoppt - Bitte Fragebogen ausfüllen")
        continue

    if waiting_for_final_press:
        client.send_message(stop, [])
        update_text_area("Ende des Programms")
        break

    if value_index < len(arrays[array_index]):
        update_text_area("Wiedergabe läuft")
        send_osc_region(int(arrays[array_index][value_index])-1, ip, port)
        value_index += 1

        if value_index >= len(arrays[array_index]):
            if array_index + 1 < len(arrays):
                switch_message_shown = True
            else:
                waiting_for_final_press = True

    elif switch_message_shown:
        switch_message_shown = False
        array_index += 1
        value_index = 0
        client.send_message(stop, [])
        update_text_area(f"Wechseln Sie zum Kopfhörer Nr {hp_order[array_index] + 1}")

def start_main_thread(text_area):
thread = threading.Thread(target=main, args=(text_area,))
thread.start()

def create_gui():
root = tk.Tk()
root.title("Hörversuch Steuerung")
text_area = tk.Text(root, wrap='word', height=20, width=100)
text_area.pack()

# Start main
start_main_thread(text_area)

root.mainloop()

if __name__ == "__main__":
create_gui()
